In [1]:
import glob, json, pathlib
from tqdm.notebook import tqdm
sanitychecks = list(glob.glob(r"D:\ig_pipeline\cad\*\*\artifacts\sanitycheck.json"))
jsons = {}
for x in tqdm(sanitychecks):
    with open(x, "r") as f:
        name = pathlib.Path(x).parts[-3]
        jsons[name] = json.load(f)

  0%|          | 0/90 [00:00<?, ?it/s]

In [2]:
jsons

{'batch-00': {'success': False,
  'errors': ["Model ID jtodol contains 'todo'."],
  'warnings': ['L-tag-ohegnp-0 has too many vertices: 21794 > 20000',
   'chicken_wire-czoiqw-0 has too many vertices: 52305 > 20000']},
 'batch-01': {'success': True,
  'errors': [],
  'warnings': ['L-shoe_rack-spdrzb-0 has too many vertices: 59456 > 20000']},
 'batch-02': {'success': False,
  'errors': ['L-alarm_clock-vqwovi-0 has no collision mesh. Create a collision mesh.'],
  'warnings': ['L-radio-vgioak-0 has too many vertices: 50123 > 20000']},
 'batch-03': {'success': True, 'errors': [], 'warnings': []},
 'batch-05': {'success': True, 'errors': [], 'warnings': []},
 'batch-06': {'success': False,
  'errors': ['L-electric_hand_mixer-prewfi-0-Mcollision element 2 has too many vertices (108 > 60)',
   'L-electric_hand_mixer-prewfi-0-Mcollision element 3 has too many vertices (108 > 60)',
   'L-electric_hand_mixer-prewfi-0-Mcollision element 7 has too many vertices (108 > 60)',
   'L-electric_hand_mix

In [3]:
total = len(jsons)
failed = {name for name, x in jsons.items() if x["errors"]}
total, len(failed)

(90, 53)

In [4]:
from collections import defaultdict
import re

warning_re = re.compile(r"(.*) has too many vertices: (\d+) > 20000")
warn_objs = defaultdict(dict)
for name, x in jsons.items():
    for w in x["warnings"]:
        m = warning_re.fullmatch(w)
        if m:
            warn_objs[name][m.group(1)] = int(m.group(2))
            
print(warn_objs)

defaultdict(<class 'dict'>, {'batch-00': {'L-tag-ohegnp-0': 21794, 'chicken_wire-czoiqw-0': 52305}, 'batch-01': {'L-shoe_rack-spdrzb-0': 59456}, 'batch-02': {'L-radio-vgioak-0': 50123}, 'batch-06': {'zipper-qwsvid-0': 27426}, 'legacy_batch-01': {'swivel_chair-fqbnuk-0-base_link': 36405, 'swivel_chair-lizuow-0-base_link': 25743, 'swivel_chair-lxupys-0-base_link': 35465, 'swivel_chair-mqxrim-0-base_link': 21060, 'swivel_chair-opkoam-0-base_link': 38343}, 'legacy_batch-02': {'table_lamp-pmekkr-0-arm-body-R-lower': 23529, 'table_lamp-pmekkr-0-head-neck-R-lower': 44331, 'table_lamp-pmekkr-0-arm-body-R-upper': 23529, 'table_lamp-pmekkr-0-head-neck-R-upper': 44331, 'table_lamp-pxdhzm-0-head-arm-R-lower': 33474, 'table_lamp-pxdhzm-0-head-arm-R-upper': 33474}, 'legacy_batch-03': {'desk-egzdgz-0-base_link': 20943, 'dishwasher-znfvmj-0-link_0-base_link-P-lower': 31812, 'dishwasher-znfvmj-0-link_1-base_link-P-lower': 31812, 'dishwasher-znfvmj-0-link_0-base_link-P-upper': 31812, 'dishwasher-znfvmj-

In [102]:
# Worst offenders
sorted((cnt, obj) for warn_target in warn_objs.values() for obj, cnt in warn_target.items())

[(20082, 'barbecue_sauce_bottle-gfxrnj-0'),
 (20082, 'catsup_bottle-qfvqfm-0'),
 (20082, 'chocolate_sauce_bottle-yegrkf-0'),
 (20082, 'glaze_bottle-zdxagk-0'),
 (20082, 'hot_sauce_bottle-qvpthd-0'),
 (20082, 'mustard_bottle-lgxfyv-0'),
 (20082, 'salsa_bottle-kydilb-0'),
 (20082, 'soy_sauce_bottle-saujjl-0'),
 (20082, 'squeeze_bottle-uzmhdn-0'),
 (20082, 'sugar_syrup_bottle-kdlbbq-0'),
 (20082, 'vanilla_bottle-drevku-0'),
 (20082, 'white_sauce_bottle-gtwngf-0'),
 (20082, 'wine_sauce_bottle-vqtevv-0'),
 (20279, 'pot_plant-kxmvco-0-base_link'),
 (20943, 'desk-egzdgz-0-base_link'),
 (21060, 'swivel_chair-mqxrim-0-base_link'),
 (21794, 'L-tag-ohegnp-0'),
 (22097, 'L-garden_chair-deuron-0'),
 (22097, 'L-garden_chair-deuron-1'),
 (22560, 'room_light-fxtqtn-0'),
 (22560, 'room_light-fxtqtn-1'),
 (22740, 'fridge-jtqazu-0-base_link'),
 (22956, 'L-pedestal_table-ifmhpn-0'),
 (22956, 'L-pedestal_table-ifmhpn-1'),
 (22998, 'C-stockpot-gxiqbw-0'),
 (22998, 'C-stockpot-gxiqbw-1'),
 (23465, 'floor_lam

In [103]:
print("\n".join(f'  "{x}",' for x in failed))

  "Benevolence_1_int",
  "hall_arch_wood",
  "restaurant_cafeteria",
  "Beechwood_0_garden",
  "restaurant_brunch",
  "school_biology",
  "Pomaria_2_int",
  "grocery_store_asian",
  "hotel_gym_spa",
  "commercial_kitchen_fire_extinguisher",
  "public_restroom_blue",
  "Pomaria_1_int",
  "legacy_batch-04",
  "school_geography",
  "school_gym",
  "Rs_int",
  "batch-11",
  "legacy_batch-03",
  "Beechwood_1_int",
  "Benevolence_2_int",
  "house_double_floor_lower",
  "Merom_1_int",
  "Ihlen_1_int",
  "public_restroom_futuristic",
  "Beechwood_0_int",
  "Benevolence_0_int",
  "grocery_store_half_stocked",
  "Merom_0_garden",
  "restaurant_urban",
  "batch-12",
  "office_cubicles_left",
  "office_vendor_machine",
  "restaurant_hotel",
  "legacy_batch-05",
  "house_single_floor",
  "legacy_batch-06",
  "restaurant_asian",
  "school_computer_lab_and_infirmary",
  "Wainscott_0_int",
  "grocery_store_convenience",
  "house_double_floor_upper",
  "office_large",
  "office_cubicles_right",
  "Wain

In [104]:
print("dvc unprotect", " ".join(f"cad/objects/{x}/processed.max" for x in sorted(failed)))

dvc unprotect cad/objects/Beechwood_0_garden/processed.max cad/objects/Beechwood_0_int/processed.max cad/objects/Beechwood_1_int/processed.max cad/objects/Benevolence_0_int/processed.max cad/objects/Benevolence_1_int/processed.max cad/objects/Benevolence_2_int/processed.max cad/objects/Ihlen_0_int/processed.max cad/objects/Ihlen_1_int/processed.max cad/objects/Merom_0_garden/processed.max cad/objects/Merom_0_int/processed.max cad/objects/Merom_1_int/processed.max cad/objects/Pomaria_0_garden/processed.max cad/objects/Pomaria_0_int/processed.max cad/objects/Pomaria_1_int/processed.max cad/objects/Pomaria_2_int/processed.max cad/objects/Rs_garden/processed.max cad/objects/Rs_int/processed.max cad/objects/Wainscott_0_garden/processed.max cad/objects/Wainscott_0_int/processed.max cad/objects/Wainscott_1_int/processed.max cad/objects/batch-00/processed.max cad/objects/batch-02/processed.max cad/objects/batch-06/processed.max cad/objects/batch-08/processed.max cad/objects/batch-11/processed.

In [9]:
from collections import Counter
import re
error_templates = [
    # "Category (.*) for object .* does not exist on spreadsheet.",
    "(.*) has too many vertices: .* > .*",
    # "(.*) has bad name.",
    # ".*(http.client.RemoteDisconnected|google.auth.exceptions.TransportError).*",
    # "(.*) has negative object offset scale.*",
    "(.*) has scale that is not 1. Reset scale.",
    ".*No instance ID 0 instance of (.*)",
    "All instances of (.*) do not have contiguous instance IDs. Missing: .*",
    # "(.*) has disallowed type.*",
    # "Light object (.*) should not have object offset rotation. Reset pivot.",
    "(.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.",
    "(.*) has different pivot offset rotation \(by (.*)\). Match pivots on each instance.",
    # "(.*) has different shear. Match scaling axes on each instance.",
    "Model ID (.*) contains 'todo'.",
    "(.*) should not have an upper side.",
    "(.*) should have an upper side.",
    "(.*) has different vertex count than recorded in provider.*",
    "(.*) has different face count than recorded in provider.*",
    "(.*) has no collision mesh. Create a collision mesh.",
    "(.*) has different UV unwrapping than recorded. Reunwrap the object.",
    "Inconsistent link sets within model (.*)",
    "(.*) element .* has too many vertices .*"
]
error_matches = [set() for _ in error_templates]
error_matches_by_file = [defaultdict(set) for _ in error_templates]
unknown_error_files = []
for name, f in jsons.items():
    unknown_error = False
    for error in f["errors"] + f["warnings"]:
        for i, tmpl in enumerate(error_templates):
            match = re.fullmatch(tmpl, error, re.S)
            if match is not None:
                error_matches[i].add(match.group(1))
                error_matches_by_file[i][name].add(match.group(1))

                break
        else:
            print(error)
            unknown_error = True
    if unknown_error:
        unknown_error_files.append(name)
        
print("\nCounts")
for objs, tmpl in zip(error_matches, error_templates):
    print(f"{tmpl}: {len(objs)}")

print("\n\nCounts by file")
for tmpl, all_matches, matches_by_file in zip(error_templates, error_matches, error_matches_by_file):
    if not all_matches:
        continue
    print(f"\n{tmpl}")
    for name, matches in sorted(matches_by_file.items(), key=lambda x: x[1], reverse=True):
        if not matches:
            break
        print(f"  {name}: {len(matches)}")

Bad object B-bidet-lcyvov-0 should not have children.
Meta link bidet-lcyvov-0-Mfluidsink_0 base name doesnt match parent B-bidet-lcyvov-0
Meta link bidet-lcyvov-0-Mtogglebutton_0 base name doesnt match parent B-bidet-lcyvov-0
Meta link bidet-lcyvov-0-Mfluidsource_0 base name doesnt match parent B-bidet-lcyvov-0

Counts
(.*) has too many vertices: .* > .*: 123
(.*) has scale that is not 1. Reset scale.: 90
.*No instance ID 0 instance of (.*): 0
All instances of (.*) do not have contiguous instance IDs. Missing: .*: 1
(.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.: 55
(.*) has different pivot offset rotation \(by (.*)\). Match pivots on each instance.: 15
Model ID (.*) contains 'todo'.: 7
(.*) should not have an upper side.: 1
(.*) should have an upper side.: 0
(.*) has different vertex count than recorded in provider.*: 35
(.*) has different face count than recorded in provider.*: 35
(.*) has no collision mesh. Create a collision mesh.: 16
(.*) has

In [106]:
vertex_issues = error_matches_by_file[1]
vertex_issues_unpacked = {}
for filename, matches in vertex_issues.items():
    unique_ids = {match.group(1) for match in matches}
    vertex_issues_unpacked[filename] = unique_ids

for filename, ids in sorted(vertex_issues_unpacked.items(), key=lambda x: len(x[1]), reverse=True):
  print(f"{filename}: {len(ids)}. ({', '.join(list(ids)[:5])}...)")

office_large: 10. (immwzb, ojceew, juwaoh, rvunhj, jhymlr...)
school_biology: 10. (aknswb, gemgfz, xxipyh, ewqpfv, khxpgc...)
office_cubicles_left: 9. (immwzb, slgzfc, ncbfwr, rvunhj, hrdeys...)
school_computer_lab_and_infirmary: 9. (aknswb, gemgfz, exaqtm, rwmofs, xxipyh...)
school_chemistry: 8. (aknswb, tqyiso, gemgfz, xxipyh, ewqpfv...)
school_geography: 8. (aknswb, gemgfz, ehwmol, ewqpfv, khxpgc...)
office_cubicles_right: 5. (immwzb, xqxiyb, rvunhj, jhymlr, oztrra...)
school_gym: 5. (xxipyh, nwlbit, rgmujm, yhthgr, uztisk...)
commercial_kitchen_fire_extinguisher: 4. (ttmejh, pkdnbu, jhymlr, jtqazu...)
hotel_gym_spa: 4. (ceddpg, nfrbch, aapttl, adejun...)
house_single_floor: 4. (xsuyua, gjrero, kfmkbm, dszchb...)
commercial_kitchen_pans: 3. (ttmejh, pkdnbu, jtqazu...)
restaurant_cafeteria: 3. (juwaoh, lsyzkh, jtqazu...)
hotel_suite_small: 2. (ftbsdt, aapttl...)
house_double_floor_lower: 2. (ffitak, supuws...)
restaurant_diner: 2. (juwaoh, olgoza...)
hotel_suite_large: 1. (aapttl...)

In [107]:
{err: examples for err, examples in zip(error_templates, error_counts)}

{'(.*)-(.*) has different vertex count than recorded in provider.*': 441,
 '(.*)-(.*) has different face count than recorded in provider.*': 428,
 '(.*) has no collision mesh. Create a collision mesh.': 16,
 '(.*) has different UV unwrapping than recorded. Reunwrap the object.': 0,
 'Inconsistent link sets within model (.*)': 101}